In [98]:
import pandas as pd
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
import numpy as np

In [29]:
df_train = pd.read_csv("../data_preprocess/train_preprocess.csv", sep=",", header=0, na_values=['?', '', 'NA'])
# Cargamos csv con los datos de test
df_test = pd.read_csv("../data_preprocess/test_preprocess.csv", sep=",", header=0, na_values=['?', '', 'NA'])


In [31]:
df_train

,ID,X1,X4,X5,X7,X9,X13,X14,X15,X16,...,X25,X27,X30,X32,X34,X35,X36,X38,X39,RATE
0,102,-0.283105,-0.294141,-0.216731,-0.281493,1.398353,-0.434783,-0.455000,0.455889,-0.411449,...,0,-1.763188,0,-0.128418,-0.372249,-0.123599,-0.398377,2.864004,-1.493411,B
1,343,0.544414,0.773519,0.849708,0.479761,0.980651,0.086957,1.215000,0.117917,0.075134,...,1,0.503571,0,1.172363,0.577876,0.547511,0.172548,0.447551,0.286241,B
2,407,-0.109076,-0.182473,-0.165846,-0.000198,-0.310319,0.434783,-0.023889,0.077177,0.132379,...,0,-0.323889,0,0.312988,0.023685,0.606281,-0.260056,-0.246029,-0.581444,B
3,950,-0.142328,-0.170376,-0.160334,-0.089851,-0.009279,0.521739,0.451667,-0.587864,-0.100179,...,0,-0.652845,0,0.380371,-0.150597,-0.084419,0.354622,1.829387,-0.269900,D
4,848,-0.248911,-0.301795,-0.314106,-0.194442,-0.252107,-0.608696,0.019444,-0.126237,0.354204,...,0,-0.386547,0,0.637207,0.081308,0.565182,0.422371,-0.551973,-0.319979,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,89,0.538190,0.617877,0.188801,0.632552,-0.171094,1.565217,-0.627222,0.324200,0.393560,...,0,-0.162635,0,-0.395996,-0.921962,-1.023932,-0.772406,-0.556728,-0.077491,B
902,571,-0.310287,-0.314977,-0.357120,-0.321784,-0.551324,-0.782609,-2.173889,0.358342,-1.649374,...,0,-1.068417,0,-1.835449,-0.948475,-0.761183,-0.549400,1.338374,-0.798629,C
903,953,3.470347,0.830079,2.029403,7.164578,0.278583,0.739130,-0.223889,-2.251327,3.191413,...,0,-0.041926,-1,-1.253418,-0.336309,-0.464607,-0.245236,0.029291,-0.187665,D
904,274,-0.029073,-0.125599,-0.283810,-0.229721,-0.476467,-0.521739,-2.563889,0.978626,-1.499106,...,0,-1.537434,0,-2.270996,-2.246222,-1.987983,0.368031,0.017309,-1.425936,D


In [35]:
df_train.iloc[:,1:25]

,X1,X4,X5,X7,X9,X13,X14,X15,X16,X18,...,X24,X25,X27,X30,X32,X34,X35,X36,X38,X39
0,-0.283105,-0.294141,-0.216731,-0.281493,1.398353,-0.434783,-0.455000,0.455889,-0.411449,-0.437620,...,0.0,0,-1.763188,0,-0.128418,-0.372249,-0.123599,-0.398377,2.864004,-1.493411
1,0.544414,0.773519,0.849708,0.479761,0.980651,0.086957,1.215000,0.117917,0.075134,0.119002,...,1.0,1,0.503571,0,1.172363,0.577876,0.547511,0.172548,0.447551,0.286241
2,-0.109076,-0.182473,-0.165846,-0.000198,-0.310319,0.434783,-0.023889,0.077177,0.132379,0.142035,...,-1.0,0,-0.323889,0,0.312988,0.023685,0.606281,-0.260056,-0.246029,-0.581444
3,-0.142328,-0.170376,-0.160334,-0.089851,-0.009279,0.521739,0.451667,-0.587864,-0.100179,-0.095969,...,2.0,0,-0.652845,0,0.380371,-0.150597,-0.084419,0.354622,1.829387,-0.269900
4,-0.248911,-0.301795,-0.314106,-0.194442,-0.252107,-0.608696,0.019444,-0.126237,0.354204,0.422265,...,1.0,0,-0.386547,0,0.637207,0.081308,0.565182,0.422371,-0.551973,-0.319979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,0.538190,0.617877,0.188801,0.632552,-0.171094,1.565217,-0.627222,0.324200,0.393560,0.383877,...,2.0,0,-0.162635,0,-0.395996,-0.921962,-1.023932,-0.772406,-0.556728,-0.077491
902,-0.310287,-0.314977,-0.357120,-0.321784,-0.551324,-0.782609,-2.173889,0.358342,-1.649374,-1.266795,...,0.0,0,-1.068417,0,-1.835449,-0.948475,-0.761183,-0.549400,1.338374,-0.798629
903,3.470347,0.830079,2.029403,7.164578,0.278583,0.739130,-0.223889,-2.251327,3.191413,2.687140,...,-2.0,0,-0.041926,-1,-1.253418,-0.336309,-0.464607,-0.245236,0.029291,-0.187665
904,-0.029073,-0.125599,-0.283810,-0.229721,-0.476467,-0.521739,-2.563889,0.978626,-1.499106,-0.944338,...,1.0,0,-1.537434,0,-2.270996,-2.246222,-1.987983,0.368031,0.017309,-1.425936


In [106]:
sm=SMOTE()
x_over, y_over=sm.fit_resample(df_train.iloc[:,1:25],df_train.iloc[:,25])

In [107]:
y_over.value_counts()

RATE
B    386
D    386
C    386
A    386
Name: count, dtype: int64

In [101]:
parameters = {'intercept_scaling':(0.1,0.25,0.5,1.0),'penalty':('l2',None),'C':(0.1,0.5,1,1.5,2)}
lr=LogisticRegression(max_iter=100000,solver='lbfgs',class_weight=None)
clf=GridSearchCV(lr,parameters,scoring='accuracy')
clf.fit(x_over,y_over)

/home/moisestellezfrancisco/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/moisestellezfrancisco/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/moisestellezfrancisco/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/moisestellezfrancisco/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/moisestellezfrancisco/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/home/moisestel

GridSearchCV(estimator=LogisticRegression(max_iter=100000),
             param_grid={'C': (0.1, 0.5, 1, 1.5, 2),
                         'intercept_scaling': (0.1, 0.25, 0.5, 1.0),
                         'penalty': ('l2', None)},
             scoring='accuracy')

In [102]:
clf.best_params_

{'C': 0.1, 'intercept_scaling': 0.1, 'penalty': None}

In [103]:
clf.scorer_

make_scorer(accuracy_score)

In [104]:
clf.best_score_

0.6757921342634718

In [96]:
pred2=clf.best_estimator_.predict(df_test.iloc[:,1:40])
if(not(np.array_equal(pred,pred2))):
    print("OJO CUIDADO QUE SON DIFERENTES")
    print(pred2)
pred=pred2

In [82]:
pred=clf.best_estimator_.predict(df_test.iloc[:,1:40])
results={'ID':df_test.iloc[:,0], 'RATE': pred}
df_submission=pd.DataFrame(data=results)
df_submission.to_csv("SubmissionSMOTE.csv",index=False)

In [30]:
results={'ID':df_test.iloc[:,0], 'RATE': pred}

In [32]:
df_submission=pd.DataFrame(data=results)

In [35]:
df_submission.to_csv("Submission.csv",index=False)

In [53]:
lr=LogisticRegression(max_iter=100000,class_weight='balanced',penalty='l2',solver='saga')
lr.fit(df_train.iloc[:,1:39],df_train.iloc[:,40])

LogisticRegression(class_weight='balanced', max_iter=100000, solver='saga')

In [26]:
def preprape_submission(classifier):
    pred=classifier.predict(df_test.iloc[:,1:40])
    results={'ID':df_test.iloc[:,0], 'RATE': pred}
    df_submission=pd.DataFrame(data=results)
    df_submission.to_csv("Submission.csv",index=False)

In [27]:
preprape_submission(clf)

FileNotFoundError: [Errno 2] No such file or directory: 'Submission.csv'

In [18]:
clf.predict_proba(df_test.iloc[:,1:40])

array([[0.08874277, 0.41487151, 0.26709393, 0.22929179],
       [0.07550051, 0.40453913, 0.30378234, 0.21617802],
       [0.06480887, 0.50562   , 0.25276237, 0.17680877],
       ...,
       [0.08855152, 0.46616364, 0.21398749, 0.23129735],
       [0.08136342, 0.40460633, 0.2595434 , 0.25448685],
       [0.09882603, 0.48088424, 0.22366756, 0.19662217]])

In [89]:
df_train.iloc[:,40].value_counts()

RATE
B    361
C    224
D    199
A     68
Name: count, dtype: int64

In [32]:
def test_model(model):
    cv=RepeatedStratifiedKFold()
    n_scores=cross_val_score(model,df_train.iloc[:,1:40],df_train.iloc[:,40],scoring="accuracy",cv=cv,n_jobs=-1)
    print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [41]:
parameters = {'penalty': ('l2','l1',None),'multi_class': ('ovr','multinomial'),'class_weight':('balanced',None)}
lr=LogisticRegression(max_iter=100000,solver='lbfgs',penalty='l2',multi_class='ovr',class_weight=None)

In [43]:
lr.fit(df_train.iloc[:,1:40],df_train.iloc[:,40])

LogisticRegression(max_iter=100000, multi_class='ovr')

In [44]:
pred=lr.predict(df_test.iloc[:,1:40])
pred

array(['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'D', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'C', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'C', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'C', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'C',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B',
       'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B